In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('data/data_v5.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,Year,t1_general_traditional_WIN%,t1_general_traditional_PTS,t1_general_traditional_FGM,t1_general_traditional_FG%,t1_general_traditional_3PM,t1_general_traditional_3P%,t1_general_traditional_FTM,...,t2_clutch-opponent_Opponent's Steals,t2_clutch-opponent_Opponent's Blocks,t2_clutch-opponent_Opponent's Blocked Field Goal Attempts,t2_clutch-opponent_Opponent's Personal Fouls,t2_clutch-opponent_Opponent's Personal Fouls Drawn,t2_clutch-opponent_Opponent's Points,t2_TEAM,t2_seed,t2_series_wins,outcome
0,0,0,2001,56.1,91.9,34.1,43.1,4.1,34.6,19.5,...,0.6,0.5,0.3,2.7,2.1,7.4,Miami Heat,3,0,1
1,1,1,2001,63.4,100.7,38.0,45.8,6.9,37.9,18.0,...,0.6,0.4,0.5,2.5,3.1,8.8,Orlando Magic,7,1,1
2,2,2,2001,50.0,92.6,34.5,44.0,4.8,34.2,18.8,...,0.6,0.5,0.6,3.3,2.5,8.2,Philadelphia 76ers,1,3,0
3,3,3,2001,58.5,88.7,33.6,44.4,4.8,35.1,16.8,...,0.5,0.4,0.4,2.2,2.6,7.6,Toronto Raptors,5,3,0
4,4,4,2001,64.6,100.5,37.6,45.9,6.3,38.1,18.9,...,0.5,0.2,0.3,2.9,2.2,6.9,Utah Jazz,4,2,1


In [3]:
df.drop(columns=['Unnamed: 0.1','Unnamed: 0', 'Year','t1_TEAM','t2_TEAM','t1_series_wins','t2_series_wins'],inplace=True)
X = df.drop(columns=['outcome']).to_numpy()
y = df['outcome'].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(X,y)

In [4]:
x_train.shape, x_test.shape

((247, 220), (83, 220))

In [4]:
def accuracy(y1,y2):
    cor = 0
    for i in range(y1.shape[0]):
        if y1[i]==y2[i]:
            cor+=1
    return cor/y1.shape[0]

In [5]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
ypreds = lr.predict(x_test)
accuracy(ypreds,y_test)

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7831325301204819

In [7]:
coefs_lr = lr.coef_

In [8]:
coefs_lr.shape

(1, 218)

In [9]:
x_train.shape

(247, 218)

In [10]:
intercept = lr.intercept_

In [13]:
intercept = intercept[0]

In [14]:
def return_class(x):
    #expect np array for x
    denom = 1 + np.exp(-1*(np.dot(x,coefs_lr.reshape(-1))+intercept))
    prob = 1/denom
    if prob > 0.5:
        return 1
    return 0

In [15]:
y_preds = np.zeros(x_test.shape[0]).reshape(-1)
for i in range(x_test.shape[0]):
    y_preds[i] = return_class(x_test[i,:].reshape(-1))
accuracy(y_preds,y_test)

0.7831325301204819

In [16]:
np.savetxt('coefs_lr.csv',coefs_lr,delimiter=',')
np.savetxt('intercept_lr.csv',lr.intercept_,delimiter=',')

In [7]:
#73% accuracy for Logistic Regression on the test set; 
#let's determine the average performance across different train/test splits


In [6]:
acc_list = []
for i in range(10):
    x_train, x_test, y_train, y_test = train_test_split(X,y)
    lr = LogisticRegression()
    lr.fit(x_train, y_train)
    ypreds = lr.predict(x_test)
    acc_list.append(accuracy(ypreds,y_test))
print(sum(acc_list)/len(acc_list))
    

0.7542168674698796


/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [ ]:
#about 76% accuracy 

In [9]:
#let's try using Bagging Logistic Regression as a classifier

In [7]:
from sklearn.ensemble import BaggingClassifier

In [9]:
bag = BaggingClassifier(estimator=LogisticRegression())
x_train, x_test, y_train, y_test = train_test_split(X,y)
bag.fit(x_train,y_train)
accuracy(bag.predict(x_test),y_test)

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

0.7469879518072289

In [14]:
#close to 75% accuracy!

In [15]:
bag.estimator

LogisticRegression()

In [19]:
lrs_in_bag = bag.estimators_
for m in lrs_in_bag:
    print(m.coef_)
    break

[[ 2.12278460e-01  2.96656244e-02  1.23965586e-02  4.56135545e-02
   8.34098191e-04  1.85022885e-02  4.60692222e-03 -5.62959362e-02
  -1.85654038e-02  1.65976448e-02 -2.85932221e-03  3.03005367e-02
  -8.74273536e-03  4.60940580e-03  2.10964331e-02 -1.13588610e-02
  -2.75599799e-02 -1.19243035e-02  7.76759502e-02  6.09683800e-02
  -1.67006940e-02  7.89481699e-02  7.23938284e-02  2.60815949e-03
   2.67522669e-02 -1.49201674e-02  1.97045188e-02  1.29038964e-02
  -4.86735201e-03  4.71544536e-02  3.95299342e-02 -1.97046743e-02
   8.10301697e-02 -3.63260388e-03 -8.98977089e-03 -5.93992527e-03
  -1.25894307e-02 -4.49971350e-02 -2.09768156e-02 -1.52075966e-02
   4.26688750e-03 -4.39820254e-02 -1.21262234e-02  3.33059723e-02
  -1.23451314e-02 -2.65935263e-02 -3.82932571e-02 -2.80196218e-02
  -1.52563100e-02 -5.59411755e-03 -1.13588610e-02  2.10964331e-02
  -6.00628565e-03 -2.75599799e-02 -4.81481960e-02 -4.18641126e-02
  -2.85400414e-02 -4.90928293e-02 -9.78077241e-03 -1.82240865e-02
  -7.48210

In [2]:
#let's work with the reduced feature dataset
x_train_df = pd.read_csv('data/x_train.csv')
x_train_df.head()

,Unnamed: 0,t1_general_traditional_WIN%,t1_general_traditional_3PM,t1_general_traditional_+/-,t1_general_advanced_NetRtg,t1_general_advanced_AST,t1_general_advanced_PIE,t1_general_opponent_Opponent's 3 Point Field Goal Percentage,t1_general_opponent_Opponent's Free Throw Percentage,t1_general_opponent_Opponent's Turnovers,...,t2_general_traditional_+/-,t2_general_advanced_OffRtg,t2_general_advanced_NetRtg,t2_general_advanced_POSS,t2_general_opponent_Opponent's 3 Point Field Goals Made,t2_general_opponent_Opponent's Rebounds,t2_clutch-traditional_3 Point Field Goals Made,t2_clutch-traditional_Free Throw Percentage,t2_clutch-opponent_Opponent's Field Goals Made,t2_seed
0,0,54.9,5.9,1.6,1.7,17.1,51.4,35.3,73.9,16.4,...,8.4,108.6,9.2,7440.0,4.5,39.1,0.6,66.2,2.7,3.0
1,1,73.2,5.1,6.1,6.7,14.6,55.1,36.1,75.2,14.0,...,5.5,110.2,5.4,8067.0,6.3,45.9,0.7,82.1,3.0,2.0
2,2,73.2,10.0,5.4,5.9,19.5,54.0,34.1,76.4,16.1,...,0.7,103.0,0.6,7844.0,8.0,41.7,0.5,70.6,3.5,5.0
3,3,64.6,5.5,4.2,4.8,17.3,53.3,33.9,74.0,14.7,...,0.8,104.1,0.8,7465.0,5.6,37.6,0.2,69.1,2.2,8.0
4,4,52.4,12.9,1.6,1.6,18.1,50.9,36.4,79.2,12.8,...,4.5,113.0,4.5,7987.0,13.0,41.7,0.9,78.0,2.0,1.0


In [3]:
x_train_df.drop(columns=['Unnamed: 0'],inplace=True)
x_test_df = pd.read_csv('data/x_test.csv').drop(columns=['Unnamed: 0'])
x_test_df.head()

,t1_general_traditional_WIN%,t1_general_traditional_3PM,t1_general_traditional_+/-,t1_general_advanced_NetRtg,t1_general_advanced_AST,t1_general_advanced_PIE,t1_general_opponent_Opponent's 3 Point Field Goal Percentage,t1_general_opponent_Opponent's Free Throw Percentage,t1_general_opponent_Opponent's Turnovers,t1_clutch-traditional_Win Percentage,...,t2_general_traditional_+/-,t2_general_advanced_OffRtg,t2_general_advanced_NetRtg,t2_general_advanced_POSS,t2_general_opponent_Opponent's 3 Point Field Goals Made,t2_general_opponent_Opponent's Rebounds,t2_clutch-traditional_3 Point Field Goals Made,t2_clutch-traditional_Free Throw Percentage,t2_clutch-opponent_Opponent's Field Goals Made,t2_seed
0,61.0,6.0,3.8,3.8,16.2,52.0,32.9,73.4,15.3,56.3,...,-0.5,108.7,-0.4,7870.0,7.3,43.0,0.7,78.8,3.3,7.0
1,72.0,5.9,7.4,8.3,17.8,55.2,33.2,74.4,13.5,55.3,...,0.4,105.1,0.5,7536.0,6.8,39.2,0.3,75.3,2.3,7.0
2,69.5,7.9,4.2,4.4,18.7,53.8,34.3,75.0,13.6,68.0,...,1.5,107.3,1.6,7359.0,6.5,39.7,0.5,78.4,2.4,6.0
3,69.5,6.4,6.1,6.3,17.0,54.1,33.5,76.5,13.6,60.5,...,0.9,105.4,1.0,7387.0,6.8,39.3,0.4,76.7,2.6,7.0
4,64.6,12.0,2.6,2.6,18.8,51.4,33.2,76.9,13.9,65.3,...,0.4,106.5,0.3,7919.0,8.9,43.2,0.6,91.7,2.5,8.0


In [4]:
y_train_df = pd.read_csv('data/y_train.csv')['outcome']
y_train_df.head()

0    0
1    1
2    1
3    1
4    0
Name: outcome, dtype: int64

In [5]:
y_test_df = pd.read_csv('data/y_test.csv')['outcome']

In [8]:
bag = BaggingClassifier(estimator=LogisticRegression())
bag.fit(x_train_df.to_numpy(),y_train_df.to_numpy())
accuracy(bag.predict(x_test_df.to_numpy()),y_test_df.to_numpy())

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

0.7727272727272727

In [9]:
#about 77% accuracy

In [10]:
#let's work with v2 of reduced dataset

In [11]:
x_train_df = pd.read_csv('data/x_train_v2.csv')
x_train_df.head()

,Unnamed: 0,t1_general_traditional_WIN%,t2_general_traditional_WIN%,t1_general_traditional_PTS,t2_general_traditional_PTS,t1_general_traditional_BLK,t2_general_traditional_BLK,t1_general_traditional_+/-,t2_general_traditional_+/-,t1_general_advanced_NetRtg,...,t1_general_advanced_DefRtg,t2_general_advanced_DefRtg,t1_general_advanced_POSS,t2_general_advanced_POSS,t1_general_defense_OPP_PTS_2ND_CHANCE,t2_general_defense_OPP_PTS_2ND_CHANCE,t1_clutch-traditional_Field Goal Percentage,t2_clutch-traditional_Field Goal Percentage,t1_clutch-traditional_Assists,t2_clutch-traditional_Assists
0,0,67.1,68.3,110.1,95.4,6.3,4.1,5.0,4.8,5.0,...,107.0,100.7,8058.0,7377.0,15.4,10.5,46.0,50.0,1.5,1.8
1,1,64.6,56.1,101.7,97.7,5.0,4.7,4.7,1.7,4.9,...,106.0,101.9,7515.0,7698.0,14.0,11.1,40.4,40.1,1.2,1.5
2,2,58.3,65.3,112.4,114.0,4.3,4.1,2.3,6.2,2.3,...,112.3,110.6,7062.0,7036.0,13.1,11.8,45.1,42.5,1.2,0.9
3,3,80.5,46.3,102.9,98.9,5.4,6.7,7.9,-1.5,8.2,...,103.2,104.2,7572.0,7909.0,14.7,14.6,48.8,41.9,2.2,1.5
4,4,80.5,72.0,100.5,97.5,4.6,5.8,10.3,7.4,10.8,...,98.1,101.5,7575.0,7279.0,12.5,11.4,39.1,41.3,1.2,1.3


In [12]:
x_train_df.drop(columns=['Unnamed: 0'],inplace=True)
x_test_df = pd.read_csv('data/x_test_v2.csv').drop(columns=['Unnamed: 0'])
x_test_df.head()

,t1_general_traditional_WIN%,t2_general_traditional_WIN%,t1_general_traditional_PTS,t2_general_traditional_PTS,t1_general_traditional_BLK,t2_general_traditional_BLK,t1_general_traditional_+/-,t2_general_traditional_+/-,t1_general_advanced_NetRtg,t2_general_advanced_NetRtg,...,t1_general_advanced_DefRtg,t2_general_advanced_DefRtg,t1_general_advanced_POSS,t2_general_advanced_POSS,t1_general_defense_OPP_PTS_2ND_CHANCE,t2_general_defense_OPP_PTS_2ND_CHANCE,t1_clutch-traditional_Field Goal Percentage,t2_clutch-traditional_Field Goal Percentage,t1_clutch-traditional_Assists,t2_clutch-traditional_Assists
0,74.4,51.2,91.4,90.3,5.0,3.8,5.8,0.5,6.3,0.3,...,96.2,100.3,7311.0,7361.0,11.4,12.2,40.6,38.1,1.6,1.3
1,65.9,61.0,110.2,101.4,5.1,4.6,4.9,5.1,4.8,5.4,...,109.1,103.9,7937.0,7605.0,16.1,12.6,48.9,36.9,1.5,1.0
2,69.5,72.0,107.9,106.2,4.8,6.1,7.0,6.3,7.3,6.4,...,104.0,103.0,7954.0,7954.0,14.2,12.3,44.2,38.5,1.3,1.1
3,81.7,68.3,110.0,103.9,6.0,5.0,10.1,3.4,9.9,3.4,...,100.4,102.6,8170.0,8037.0,14.2,14.0,43.5,40.5,1.5,1.0
4,59.8,58.5,112.4,108.0,5.3,4.9,4.4,3.3,4.3,3.4,...,107.0,106.0,8282.0,8097.0,13.5,13.3,50.0,49.5,1.5,1.3


In [13]:
y_train_df = pd.read_csv('data/y_train_v2.csv')['outcome']
y_test_df = pd.read_csv('data/y_test_v2.csv')['outcome']

In [16]:
bag = BaggingClassifier(estimator=LogisticRegression())
bag.fit(x_train_df.to_numpy(),y_train_df.to_numpy())
accuracy(bag.predict(x_test_df.to_numpy()),y_test_df.to_numpy())

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

0.7121212121212122

In [ ]:
#71% accuracy .. not good

In [21]:
#let's work with v6 data
df = pd.read_csv('data/data_v6.csv')
df.head()

,general_traditional_WIN%_diff,general_traditional_PTS_diff,general_traditional_FGM_diff,general_traditional_FG%_diff,general_traditional_3PM_diff,general_traditional_3P%_diff,general_traditional_FTM_diff,general_traditional_FT%_diff,general_traditional_OREB_diff,general_traditional_DREB_diff,...,clutch-opponent_Opponent's Assists_diff,clutch-opponent_Opponent's Turnovers_diff,clutch-opponent_Opponent's Steals_diff,clutch-opponent_Opponent's Blocks_diff,clutch-opponent_Opponent's Blocked Field Goal Attempts_diff,clutch-opponent_Opponent's Personal Fouls_diff,clutch-opponent_Opponent's Personal Fouls Drawn_diff,clutch-opponent_Opponent's Points_diff,seed_diff,outcome
0,-4.9,3.0,1.2,0.1,-1.7,0.1,2.1,-1.4,2.7,2.1,...,0.4,-0.1,-0.2,-0.1,0.0,-0.1,0.1,0.5,3,1
1,11.0,3.2,1.3,2.0,0.9,1.5,0.0,7.3,-1.1,0.6,...,-0.2,-0.5,-0.1,-0.2,-0.2,-0.1,-1.0,-1.8,-5,1
2,-18.3,-2.1,-0.9,-0.7,1.6,1.6,-1.9,2.1,-1.9,-0.1,...,-0.3,-0.1,0.2,0.1,0.2,-0.5,0.2,0.4,7,0
3,1.2,-8.9,-3.6,0.7,-0.4,-1.8,-1.3,4.9,-4.2,0.0,...,0.0,0.5,0.2,0.1,0.0,0.0,0.0,-0.3,-1,0
4,0.0,3.4,1.5,-1.2,2.3,0.0,-2.0,4.2,-1.4,2.3,...,0.5,0.3,-0.2,0.2,0.1,-0.3,0.5,1.4,1,1


In [22]:
X = df.drop(columns=['outcome']).to_numpy()
y = df['outcome'].to_numpy()
x_train, x_test, y_train, y_test = train_test_split(X,y)

In [35]:
lr = LogisticRegression(penalty=None)
x_train, x_test, y_train, y_test = train_test_split(X,y)
lr.fit(x_train, y_train)
ypreds = lr.predict(x_test)
accuracy(ypreds,y_test)

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6987951807228916

In [20]:
acc_list = []
for i in range(10):
    lr = LogisticRegression(penalty=None)
    x_train, x_test, y_train, y_test = train_test_split(X,y)
    lr.fit(x_train, y_train)
    ypreds = lr.predict(x_test)
    acc_list.append(accuracy(ypreds,y_test))
print(sum(acc_list)/len(acc_list))
    

0.708433734939759


/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [ ]:
#about 72% accuracy 

In [15]:
bag = BaggingClassifier(estimator=LogisticRegression(penalty=None))
bag.fit(x_train,y_train)
accuracy(bag.predict(x_test),y_test)

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

0.7349397590361446

In [12]:
#about 80% accuracy when using bagging logistic regression with v6 data